# Shadow Art Calculator

## Introduction

This notebook demonstrates a tool for calculating the optimal time of day to create shadow-based art installations. By leveraging astronomical calculations, we can predict exactly when the sun will cast shadows of desired lengths and angles at a specific location and date.

## Setup and Dependencies


```python
# Install required packages
!pip install ephem

# Import necessary libraries
import datetime
import math
from datetime import timedelta
import ephem
```

## The Science Behind Shadow Calculation

Shadows are formed when an object blocks light from a source. The length and direction of a shadow depend on:

1. The position of the light source (the sun in our case)
2. The height of the object casting the shadow
3. The angle of the light source relative to the object

The sun's position in the sky is determined by:
- Observer's latitude and longitude
- Date and time
- Atmospheric refraction

Shadow length can be calculated using trigonometry: shadow length = object height / tan(sun altitude)

## Core Calculation Function


```python
def calculate_shadow_times(latitude, longitude, target_date, shadow_length_ratio, desired_angle=None):
    """
    Calculate times during the day when shadows will be at a specific length ratio and/or angle.
    
    Parameters:
    - latitude: float, latitude in decimal degrees (positive for North, negative for South)
    - longitude: float, longitude in decimal degrees (positive for East, negative for West)
    - target_date: datetime.date object, the date to calculate for
    - shadow_length_ratio: float, desired shadow length as a ratio of object height (e.g., 1.5 means shadow is 1.5x object height)
    - desired_angle: float or None, desired shadow angle in degrees (0 = North, 90 = East, 180 = South, 270 = West)
    
    Returns:
    - List of (datetime, shadow_length_ratio, shadow_angle) tuples that meet the criteria
    """
    # Create observer location
    observer = ephem.Observer()
    observer.lat = str(latitude)
    observer.lon = str(longitude)
    observer.elevation = 0
    
    # Create sun object
    sun = ephem.Sun()
    
    # Initialize sunrise and sunset
    observer.date = target_date
    sunrise = observer.next_rising(sun).datetime()
    sunset = observer.next_setting(sun).datetime()
    
    # Check times throughout the day
    current_time = sunrise
    results = []
    
    while current_time < sunset:
        observer.date = current_time
        sun.compute(observer)
        
        # Calculate shadow length ratio based on sun altitude
        altitude = float(sun.alt)  # Sun's altitude in radians
        altitude_degrees = math.degrees(altitude)
        
        # Skip if sun is too low (near horizon)
        if altitude_degrees < 5:
            current_time += timedelta(minutes=10)
            continue
            
        # Calculate shadow length ratio (cot(altitude))
        shadow_ratio = 1 / math.tan(altitude)
        
        # Calculate shadow direction (opposite of sun's azimuth)
        azimuth = float(sun.az)  # Sun's azimuth in radians
        shadow_angle = (math.degrees(azimuth) + 180) % 360
        
        # Check if this time meets our criteria
        length_match = abs(shadow_ratio - shadow_length_ratio) < 0.1
        angle_match = desired_angle is None or abs((shadow_angle - desired_angle) % 360) < 5
        
        if length_match and angle_match:
            results.append((current_time, shadow_ratio, shadow_angle))
        
        # Move forward in time
        current_time += timedelta(minutes=10)
    
    return results
```

## Helper Functions for Result Formatting


```python
def format_results(results):
    """Format the results into a readable string."""
    if not results:
        return "No suitable times found for the given criteria."
        
    output = "Optimal times for shadow art installation:\n"
    for time, ratio, angle in results:
        output += f"Time: {time.strftime('%H:%M')}, Shadow Length: {ratio:.2f}x object height, "
        output += f"Shadow Angle: {angle:.1f}° ({get_direction(angle)})\n"
    return output

def get_direction(angle):
    """Convert angle in degrees to cardinal direction."""
    directions = ["N", "NNE", "NE", "ENE", "E", "ESE", "SE", "SSE", 
                  "S", "SSW", "SW", "WSW", "W", "WNW", "NW", "NNW"]
    index = round(angle / 22.5) % 16
    return directions[index]
```

## Main User Interface Function


```python
def shadow_art_calculator():
    """Interactive function to calculate shadow art timing."""
    print("===== Shadow Art Calculator =====")
    print("This tool helps you find the perfect time for shadow art installations.")
    
    # Get user inputs
    try:
        latitude = float(input("Enter latitude (decimal degrees, positive for North): "))
        longitude = float(input("Enter longitude (decimal degrees, positive for East): "))
        
        date_input = input("Enter date (YYYY-MM-DD) or press Enter for today: ")
        if date_input:
            target_date = datetime.datetime.strptime(date_input, "%Y-%m-%d").date()
        else:
            target_date = datetime.datetime.now().date()
            
        shadow_length = float(input("Enter desired shadow length ratio (e.g., 1.5 for 1.5x object height): "))
        
        angle_input = input("Enter desired shadow angle in degrees (0-360, 0=North, 90=East, etc.) or press Enter for any angle: ")
        desired_angle = float(angle_input) if angle_input else None
        
        # Calculate and display results
        results = calculate_shadow_times(latitude, longitude, target_date, shadow_length, desired_angle)
        print("\n" + format_results(results))
        
        if results:
            print("\nSetup Tips:")
            print("- For vertical objects, ensure they're perfectly upright.")
            print("- Use a compass to orient your shadow receiver in the optimal direction.")
            print("- Arrive at least 30 minutes before the calculated time to set up.")
            print("- Account for elevation changes and potential obstructions.")
        
    except ValueError as e:
        print(f"Error with input: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")
```

## Example Usage


```python
# Example: Let's find shadow times for New York City on June 21st
# Location: 40.7128° N, 74.0060° W
latitude = 40.7128
longitude = -74.0060
date = datetime.datetime(2025, 6, 21).date()
shadow_ratio = 1.0  # Equal to object height
desired_angle = 270  # West-facing shadow

results = calculate_shadow_times(latitude, longitude, date, shadow_ratio, desired_angle)
print(format_results(results))
```

## Running the Interactive Calculator


```python
# Uncomment to run the interactive calculator
# shadow_art_calculator()
```

## How The Algorithm Works

1. **Initialize Location**: We create an observer at the specified latitude/longitude using the ephem library
2. **Get Sun Data**: Calculate sunrise and sunset for the target date
3. **Time Loop**: For each interval (every 10 minutes) between sunrise and sunset:
   - Calculate the sun's altitude and azimuth
   - Convert sun altitude to shadow length ratio using the cotangent function
   - Calculate shadow angle (opposite of sun's azimuth)
   - Check if current conditions match desired criteria
   - Save matching times to results list
4. **Format and Display**: Convert raw calculations to human-readable format

## Conclusion

This calculator makes precise shadow art planning possible by calculating exactly when shadows will have desired properties. This transforms shadow art from guesswork to precision planning, enabling more complex installations that rely on specific shadow characteristics.

In [1]:
import datetime
import math
from datetime import timedelta
import ephem 

def calculate_shadow_times(latitude, longitude, target_date, shadow_length_ratio, desired_angle=None):
    """
    Calculate times during the day when shadows will be at a specific length ratio and/or angle.
    
    Parameters:
    - latitude: float, latitude in decimal degrees (positive for North, negative for South)
    - longitude: float, longitude in decimal degrees (positive for East, negative for West)
    - target_date: datetime.date object, the date to calculate for
    - shadow_length_ratio: float, desired shadow length as a ratio of object height (e.g., 1.5 means shadow is 1.5x object height)
    - desired_angle: float or None, desired shadow angle in degrees (0 = North, 90 = East, 180 = South, 270 = West)
    
    Returns:
    - List of (datetime, shadow_length_ratio, shadow_angle) tuples that meet the criteria
    """
    # Create observer location
    observer = ephem.Observer()
    observer.lat = str(latitude)
    observer.lon = str(longitude)
    observer.elevation = 0
    
    # Create sun object
    sun = ephem.Sun()
    
    # Initialize sunrise and sunset
    observer.date = target_date
    sunrise = observer.next_rising(sun).datetime()
    sunset = observer.next_setting(sun).datetime()
    
    # Check times throughout the day
    current_time = sunrise
    results = []
    
    while current_time < sunset:
        observer.date = current_time
        sun.compute(observer)
        
        # Calculate shadow length ratio based on sun altitude
        altitude = float(sun.alt)  # Sun's altitude in radians
        altitude_degrees = math.degrees(altitude)
        
        # Skip if sun is too low (near horizon)
        if altitude_degrees < 5:
            current_time += timedelta(minutes=10)
            continue
            
        # Calculate shadow length ratio (cot(altitude))
        shadow_ratio = 1 / math.tan(altitude)
        
        # Calculate shadow direction (opposite of sun's azimuth)
        azimuth = float(sun.az)  # Sun's azimuth in radians
        shadow_angle = (math.degrees(azimuth) + 180) % 360
        
        # Check if this time meets our criteria
        length_match = abs(shadow_ratio - shadow_length_ratio) < 0.1
        angle_match = desired_angle is None or abs((shadow_angle - desired_angle) % 360) < 5
        
        if length_match and angle_match:
            results.append((current_time, shadow_ratio, shadow_angle))
        
        # Move forward in time
        current_time += timedelta(minutes=10)
    
    return results

def format_results(results):
    """Format the results into a readable string."""
    if not results:
        return "No suitable times found for the given criteria."
        
    output = "Optimal times for shadow art installation:\n"
    for time, ratio, angle in results:
        output += f"Time: {time.strftime('%H:%M')}, Shadow Length: {ratio:.2f}x object height, "
        output += f"Shadow Angle: {angle:.1f}° ({get_direction(angle)})\n"
    return output

def get_direction(angle):
    """Convert angle in degrees to cardinal direction."""
    directions = ["N", "NNE", "NE", "ENE", "E", "ESE", "SE", "SSE", 
                  "S", "SSW", "SW", "WSW", "W", "WNW", "NW", "NNW"]
    index = round(angle / 22.5) % 16
    return directions[index]

def shadow_art_calculator():
    """Interactive function to calculate shadow art timing."""
    print("===== Shadow Art Calculator =====")
    print("This tool helps you find the perfect time for shadow art installations.")
    
    # Get user inputs
    try:
        latitude = float(input("Enter latitude (decimal degrees, positive for North): "))
        longitude = float(input("Enter longitude (decimal degrees, positive for East): "))
        
        date_input = input("Enter date (YYYY-MM-DD) or press Enter for today: ")
        if date_input:
            target_date = datetime.datetime.strptime(date_input, "%Y-%m-%d").date()
        else:
            target_date = datetime.datetime.now().date()
            
        shadow_length = float(input("Enter desired shadow length ratio (e.g., 1.5 for 1.5x object height): "))
        
        angle_input = input("Enter desired shadow angle in degrees (0-360, 0=North, 90=East, etc.) or press Enter for any angle: ")
        desired_angle = float(angle_input) if angle_input else None
        
        # Calculate and display results
        results = calculate_shadow_times(latitude, longitude, target_date, shadow_length, desired_angle)
        print("\n" + format_results(results))
        
        if results:
            print("\nSetup Tips:")
            print("- For vertical objects, ensure they're perfectly upright.")
            print("- Use a compass to orient your shadow receiver in the optimal direction.")
            print("- Arrive at least 30 minutes before the calculated time to set up.")
            print("- Account for elevation changes and potential obstructions.")
        
    except ValueError as e:
        print(f"Error with input: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    shadow_art_calculator()

===== Shadow Art Calculator =====
This tool helps you find the perfect time for shadow art installations.
Enter latitude (decimal degrees, positive for North): 7.6551
Enter longitude (decimal degrees, positive for East): 80.1261
Enter date (YYYY-MM-DD) or press Enter for today: 2025-04-19
Enter desired shadow length ratio (e.g., 1.5 for 1.5x object height): 2
Enter desired shadow angle in degrees (0-360, 0=North, 90=East, etc.) or press Enter for any angle: 

Optimal times for shadow art installation:
Time: 02:18, Shadow Length: 2.05x object height, Shadow Angle: 261.2° (W)
Time: 10:58, Shadow Length: 2.06x object height, Shadow Angle: 98.9° (E)


Setup Tips:
- For vertical objects, ensure they're perfectly upright.
- Use a compass to orient your shadow receiver in the optimal direction.
- Arrive at least 30 minutes before the calculated time to set up.
- Account for elevation changes and potential obstructions.
